<a href="https://colab.research.google.com/github/kundajelab/feature_interactions/blob/master/zeropadding/ComputeModelPredsForEvaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%tensorflow_version 1.x

In [6]:
!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/test_simulation.simdata.gz

!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/test_toanalyze_orig_seqs.txt.gz
!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/test_toanalyze_taloff_seqs.txt.gz
!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/test_toanalyze_gataoff_seqs.txt.gz
!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/test_toanalyze_bothoff_seqs.txt.gz

!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/controltest_toanalyze_taloff_seqs.txt.gz
!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/controltest_toanalyze_gataoff_seqs.txt.gz
!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/controltest_toanalyze_bothoff_seqs.txt.gz

!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/train_toanalyze_orig_seqs.txt.gz
!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/train_toanalyze_taloff_seqs.txt.gz
!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/train_toanalyze_gataoff_seqs.txt.gz
!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/train_toanalyze_bothoff_seqs.txt.gz

!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/controltrain_toanalyze_taloff_seqs.txt.gz
!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/controltrain_toanalyze_gataoff_seqs.txt.gz
!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/controltrain_toanalyze_bothoff_seqs.txt.gz

!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/shuff_toanalyze_orig_seqs.txt.gz
!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/shuff_toanalyze_taloff_seqs.txt.gz
!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/shuff_toanalyze_gataoff_seqs.txt.gz
!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/shuff_toanalyze_bothoff_seqs.txt.gz

--2020-06-26 00:26:50--  https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/test_simulation.simdata.gz
Resolving github.com (github.com)... 140.82.118.3
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kundajelab/feature_interactions/78d4a6bb7cc34b940fa2b68dba2c7fb91141a165/zeropadding/data/test_simulation.simdata.gz [following]
--2020-06-26 00:26:50--  https://raw.githubusercontent.com/kundajelab/feature_interactions/78d4a6bb7cc34b940fa2b68dba2c7fb91141a165/zeropadding/data/test_simulation.simdata.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2103069 (2.0M) [application/octet-stream]
Saving to: ‘test_simulation.simd

In [7]:
!gunzip -f *.gz

In [8]:
!md5sum *

f02081f3704a2e58fe858986738422ae  controltest_toanalyze_bothoff_seqs.txt
bbfc0081e25ac989dccc81b3adac1f46  controltest_toanalyze_bothoff_seqs.txt.gz.1
52f8393645eb58899d9dbc1ba59c8c39  controltest_toanalyze_gataoff_seqs.txt
0b4f0376d8d7f3c12a4fb88d257a81c5  controltest_toanalyze_gataoff_seqs.txt.gz.1
72e3246fb6e7da4ea275a959b1a0f9df  controltest_toanalyze_taloff_seqs.txt
929e987ba8f78c19ae4d063540c329b7  controltest_toanalyze_taloff_seqs.txt.gz.1
2639503d51ea1163c1008844323e00f3  controltrain_toanalyze_bothoff_seqs.txt
b8ca6db6526faa57c8b68a6aa5658d8b  controltrain_toanalyze_gataoff_seqs.txt
4f0e3b9bc8d10c3ce9a3c6a8d04c82a3  controltrain_toanalyze_taloff_seqs.txt
md5sum: sample_data: Is a directory
4f6c613269dccd3e6cb59cdf8f91b90c  shuff_toanalyze_bothoff_seqs.txt
7e0165121f5b1d24d07d576555d3c27f  shuff_toanalyze_gataoff_seqs.txt
fcd89000371697a65d62ae618eaea0e9  shuff_toanalyze_orig_seqs.txt
5be0269a68c6e5538fe80c899d9497cc  shuff_toanalyze_taloff_seqs.txt
85c3688d157254921f64904e71c0

In [9]:
import numpy as np


#this is set up for 1d convolutions where examples
#have dimensions (len, num_channels) 
#the channel axis is the axis for one-hot encoding.
def one_hot_encode_along_channel_axis(sequence):
    to_return = np.zeros((len(sequence),4), dtype=np.int8)
    seq_to_one_hot_fill_in_array(zeros_array=to_return,
                                 sequence=sequence, one_hot_axis=1)
    return to_return


def seq_to_one_hot_fill_in_array(zeros_array, sequence, one_hot_axis):
    assert one_hot_axis==0 or one_hot_axis==1
    if (one_hot_axis==0):
        assert zeros_array.shape[1] == len(sequence)
    elif (one_hot_axis==1): 
        assert zeros_array.shape[0] == len(sequence)
    #will mutate zeros_array
    for (i,char) in enumerate(sequence):
        if (char=="A" or char=="a"):
            char_idx = 0
        elif (char=="C" or char=="c"):
            char_idx = 1
        elif (char=="G" or char=="g"):
            char_idx = 2
        elif (char=="T" or char=="t"):
            char_idx = 3
        elif (char=="N" or char=="n"):
            continue #leave that pos as all 0's
        else:
            raise RuntimeError("Unsupported character: "+str(char))
        if (one_hot_axis==0):
            zeros_array[char_idx,i] = 1
        elif (one_hot_axis==1):
            zeros_array[i,char_idx] = 1


def inverse_anscombe_transform(vals):
  return np.square(vals/2.0) - 3.0/8


def read_labels_and_oracle_notransform(filename):
  labels = (np.array([float(x.split("\t")[0]) for x in open(filename)]))
  oracle = (np.array([float(x.split("\t")[1]) for x in open(filename)]))
  return labels, oracle


def load_onehot_sequences(filename):
  return np.array([one_hot_encode_along_channel_axis(x.rstrip())
                   for x in open(filename)])


test_toanalyze_orig_onehot =\
  load_onehot_sequences("test_toanalyze_orig_seqs.txt")
test_toanalyze_gataoff_onehot =\
  load_onehot_sequences("test_toanalyze_gataoff_seqs.txt")
test_toanalyze_taloff_onehot =\
  load_onehot_sequences("test_toanalyze_taloff_seqs.txt")
test_toanalyze_bothoff_onehot =\
  load_onehot_sequences("test_toanalyze_bothoff_seqs.txt")

controltest_toanalyze_gataoff_onehot =\
  load_onehot_sequences("controltest_toanalyze_gataoff_seqs.txt")
controltest_toanalyze_taloff_onehot =\
  load_onehot_sequences("controltest_toanalyze_taloff_seqs.txt")
controltest_toanalyze_bothoff_onehot =\
  load_onehot_sequences("controltest_toanalyze_bothoff_seqs.txt")

train_toanalyze_orig_onehot =\
  load_onehot_sequences("train_toanalyze_orig_seqs.txt")
train_toanalyze_gataoff_onehot =\
  load_onehot_sequences("train_toanalyze_gataoff_seqs.txt")
train_toanalyze_taloff_onehot =\
  load_onehot_sequences("train_toanalyze_taloff_seqs.txt")
train_toanalyze_bothoff_onehot =\
  load_onehot_sequences("train_toanalyze_bothoff_seqs.txt")

controltrain_toanalyze_gataoff_onehot =\
  load_onehot_sequences("controltrain_toanalyze_gataoff_seqs.txt")
controltrain_toanalyze_taloff_onehot =\
  load_onehot_sequences("controltrain_toanalyze_taloff_seqs.txt")
controltrain_toanalyze_bothoff_onehot =\
  load_onehot_sequences("controltrain_toanalyze_bothoff_seqs.txt")

shuff_toanalyze_orig_onehot =\
  load_onehot_sequences("shuff_toanalyze_orig_seqs.txt")
shuff_toanalyze_gataoff_onehot =\
  load_onehot_sequences("shuff_toanalyze_gataoff_seqs.txt")
shuff_toanalyze_taloff_onehot =\
  load_onehot_sequences("shuff_toanalyze_taloff_seqs.txt")
shuff_toanalyze_bothoff_onehot =\
  load_onehot_sequences("shuff_toanalyze_bothoff_seqs.txt")


In [14]:
from google.colab import drive
drive.mount('/content/drive')

!ln -s /content/drive/My\ Drive/colab_notebook_data/feature_interactions/trained_models/zeropadding/ trained_models
!ls trained_models

#!wget https://github.com/kundajelab/feature_interactions/raw/77d29d1/av/trained_models.tgz
#!tar -xzf trained_models.tgz
#!ls trained_models/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
negcontrol_model-anscombemse_arch1_l1reg-0.0001_seed100_bemnn.h5
negcontrol_model-anscombemse_arch1_l1reg-0.0001_seed200_dmeaz.h5
negcontrol_model-anscombemse_arch1_l1reg-0.0001_seed300_vwlzj.h5
negcontrol_model-anscombemse_arch1_l1reg-0.0001_seed400_vyqvb.h5
negcontrol_model-anscombemse_arch1_l1reg-0.0001_seed500_msloc.h5
negcontrol_model-anscombemse_arch1_l1reg-0.001_seed100_tiefn.h5
negcontrol_model-anscombemse_arch1_l1reg-0.001_seed200_iwhrl.h5
negcontrol_model-anscombemse_arch1_l1reg-0.001_seed300_trosb.h5
negcontrol_model-anscombemse_arch1_l1reg-0.001_seed400_lwxxh.h5
negcontrol_model-anscombemse_arch1_l1reg-0.001_seed500_hpjfm.h5
negcontrol_model-anscombemse_arch1_l1reg-0.0_seed100_gbywe.h5
negcontrol_model-anscombemse_arch1_l1reg-0.0_seed200_brovm.h5
negcontrol_model-anscombemse_arch1_l1reg-0.0_seed300_gszmc.h5
negcontrol_model-anscombemse_arch1_l1reg

In [15]:
from collections import namedtuple
import scipy.stats


PerturbedSeqs = namedtuple("PerturbedSeqs",
  ["orig_onehot", "Aoff_onehot", "Boff_onehot", "bothoff_onehot"] )


test_toanalyze_perturbedseqs = PerturbedSeqs(
    orig_onehot=test_toanalyze_orig_onehot,
    Aoff_onehot=test_toanalyze_gataoff_onehot,
    Boff_onehot=test_toanalyze_taloff_onehot,
    bothoff_onehot=test_toanalyze_bothoff_onehot)

controltest_toanalyze_perturbedseqs = PerturbedSeqs(
    orig_onehot=test_toanalyze_orig_onehot,
    Aoff_onehot=controltest_toanalyze_gataoff_onehot,
    Boff_onehot=controltest_toanalyze_taloff_onehot,
    bothoff_onehot=controltest_toanalyze_bothoff_onehot)

train_toanalyze_perturbedseqs = PerturbedSeqs(
    orig_onehot=train_toanalyze_orig_onehot,
    Aoff_onehot=train_toanalyze_gataoff_onehot,
    Boff_onehot=train_toanalyze_taloff_onehot,
    bothoff_onehot=train_toanalyze_bothoff_onehot)

controltrain_toanalyze_perturbedseqs = PerturbedSeqs(
    orig_onehot=train_toanalyze_orig_onehot,
    Aoff_onehot=controltrain_toanalyze_gataoff_onehot,
    Boff_onehot=controltrain_toanalyze_taloff_onehot,
    bothoff_onehot=controltrain_toanalyze_bothoff_onehot)

shuff_toanalyze_perturbedseqs = PerturbedSeqs(
    orig_onehot=shuff_toanalyze_orig_onehot,
    Aoff_onehot=shuff_toanalyze_gataoff_onehot,
    Boff_onehot=shuff_toanalyze_taloff_onehot,
    bothoff_onehot=shuff_toanalyze_bothoff_onehot)


PERTURBEDSEQS_DICT = {
  "test_toanalyze_perturbedseqs": test_toanalyze_perturbedseqs,
  "controltest_toanalyze_perturbedseqs": controltest_toanalyze_perturbedseqs,
  "train_toanalyze_perturbedseqs": train_toanalyze_perturbedseqs,
  "controltrain_toanalyze_perturbedseqs": controltrain_toanalyze_perturbedseqs,
  "shuff_toanalyze_perturbedseqs": shuff_toanalyze_perturbedseqs
}


def compute_prediction(model, onehot):
  return inverse_anscombe_transform(np.squeeze(model.predict(onehot)))


def compute_preds_on_perturbedseqs(model, perturbedseqs):
  orig_onehot_preds = compute_prediction(model=model,
                                         onehot=perturbedseqs.orig_onehot)
  Aoff_onehot_preds = compute_prediction(model=model,
                                         onehot=perturbedseqs.Aoff_onehot)
  Boff_onehot_preds = compute_prediction(model=model,
                                         onehot=perturbedseqs.Boff_onehot)
  bothoff_onehot_preds = compute_prediction(model=model,
                                            onehot=perturbedseqs.bothoff_onehot)
  return {"orig_onehot_preds": orig_onehot_preds,
          "Aoff_onehot_preds": Aoff_onehot_preds,
          "Boff_onehot_preds": Boff_onehot_preds,
          "bothoff_onehot_preds": bothoff_onehot_preds}


def compute_preds(model):
  to_return = {}
  for key, perturbedseqs in PERTURBEDSEQS_DICT.items():
    to_return[key] = compute_preds_on_perturbedseqs(
                      model=model, perturbedseqs=perturbedseqs)
  return to_return
  

In [16]:
import glob
import keras
import h5py

model_files = glob.glob("trained_models/*anscombemse*.h5")
!mkdir model_preds

for model_file in model_files:
  model = keras.models.load_model(model_file)
  model_preds = compute_preds(model=model)
  preds_savefile = "model_preds/"+(model_file.split("/")[-1])
  print("File",preds_savefile)
  fh = h5py.File(preds_savefile, "w")
  for key in model_preds:
    grp = fh.create_group(key)
    for subkey in model_preds[key]:
      grp.create_dataset(subkey, data=model_preds[key][subkey])
  fh.close()

mkdir: cannot create directory ‘model_preds’: File exists

File model_preds/negcontrol_model-anscombemse_arch1_l1reg-0.0_seed100_gbywe.h5
File model_preds/negcontrol_model-anscombemse_arch2_l1reg-0.0_seed100_knxqc.h5
File model_preds/negcontrol_model-anscombemse_arch3_l1reg-0.0_seed100_ydtvz.h5
File model_preds/negcontrol_model-anscombemse_arch1_l1reg-0.0001_seed100_bemnn.h5
File model_preds/negcontrol_model-anscombemse_arch2_l1reg-0.0001_seed100_hharj.h5
File model_preds/negcontrol_model-anscombemse_arch3_l1reg-0.0001_seed100_dgbrg.h5
File model_preds/negcontrol_model-anscombemse_arch1_l1reg-0.001_seed100_tiefn.h5
File model_preds/negcontrol_model-anscombemse_arch3_l1reg-0.001_seed100_nomup.h5
File model_preds/negcontrol_model-anscombemse_arch1_l1reg-0.0_seed200_brovm.h5
File model_preds/negcontrol_model-anscombemse_arch2_l1reg-0.0_seed200_ljpxf.h5
File model_preds/negcontrol_model-anscombemse_arch3_l1reg-0.0_seed200_jzmpd.h5
File model_preds/negcontrol_model-anscombemse_arch1_l1reg-0

In [18]:
!mkdir -p /content/drive/My\ Drive/colab_notebook_data/feature_interactions/model_preds_zeropadding/
!mv model_preds/* /content/drive/My\ Drive/colab_notebook_data/feature_interactions/model_preds_zeropadding/
!ls /content/drive/My\ Drive/colab_notebook_data/feature_interactions/model_preds_zeropadding/
#from google.colab import files
#!tar -czf model_preds.tgz model_preds
#files.download("model_preds.tgz")

negcontrol_model-anscombemse_arch1_l1reg-0.0001_seed100_bemnn.h5
negcontrol_model-anscombemse_arch1_l1reg-0.0001_seed200_dmeaz.h5
negcontrol_model-anscombemse_arch1_l1reg-0.0001_seed300_vwlzj.h5
negcontrol_model-anscombemse_arch1_l1reg-0.0001_seed400_vyqvb.h5
negcontrol_model-anscombemse_arch1_l1reg-0.0001_seed500_msloc.h5
negcontrol_model-anscombemse_arch1_l1reg-0.001_seed100_tiefn.h5
negcontrol_model-anscombemse_arch1_l1reg-0.001_seed200_iwhrl.h5
negcontrol_model-anscombemse_arch1_l1reg-0.001_seed300_trosb.h5
negcontrol_model-anscombemse_arch1_l1reg-0.001_seed400_lwxxh.h5
negcontrol_model-anscombemse_arch1_l1reg-0.001_seed500_hpjfm.h5
negcontrol_model-anscombemse_arch1_l1reg-0.0_seed100_gbywe.h5
negcontrol_model-anscombemse_arch1_l1reg-0.0_seed200_brovm.h5
negcontrol_model-anscombemse_arch1_l1reg-0.0_seed300_gszmc.h5
negcontrol_model-anscombemse_arch1_l1reg-0.0_seed400_mmcjt.h5
negcontrol_model-anscombemse_arch1_l1reg-0.0_seed500_pylwm.h5
negcontrol_model-anscombemse_arch2_l1reg-0.00